# 식신 - 양재역으로 검색


In [1]:
import requests
import pandas as pd
from urllib.parse import quote


- 데이터 가져오기

In [3]:
base_url = 'https://www.siksinhot.com/search?keywords='
url =f'{base_url}{"양재역"}'
req = requests.get(url)
html=req.text


- 찾으려고 하는 데이터의 태그 찾기


In [9]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

In [11]:
lis =soup.select('div.listTy1 > ul > li')
len(lis)

12

- 여러개의 데이터중 하나를 선택해서 원하는 정보 추출

In [15]:
li =lis[0]
href=li.select_one('a')['href']
href

'/P/358859'

In [16]:
base_url='https://www.siksinhot.com'
sub_url = base_url + href
req = requests.get(sub_url)
sub_soup = BeautifulSoup(req.text,'html.parser')


In [19]:
store = sub_soup.select_one('.title')
info = store.find('h3').get_text()
info


'그릭슈바인 양재역점3.6HOT'

In [20]:
score =store.select_one('h3>strong').string
score

'3.6'

In [22]:
info.find(score)


10

In [24]:
name = info[:info.find(score)]
name

'그릭슈바인 양재역점'

In [25]:
menu = sub_soup.select('.store_info p')[1].get_text()
menu

'나이트라이프 > 맥주/호프'

In [29]:
tel = sub_soup.select_one('.p_tel  p').get_text()
tel

'02-572-6100'

In [30]:
sub_soup.select_one('.txt_adr').string

'서울특별시 서초구 강남대로 224'

- 한페이지에 있는 모든 데이터를 반복문으로 가져오기


In [ ]:
name_list, score_list, menu_list, tel_list, addr_list=[], [], [], [], []
for li in enumerate(lis):
    href = li.select_one('a')['href']
    url = base_url+href
    req = requests.get(url)
    sub_soup = BeautifulSoup(req.text,'html.parser')

    store = sub_soup.select_one('.title')
    info = store.find('h3').get_text()

